# Library

In [20]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

# Folder

In [2]:
drive = "D"

In [3]:
ods_fold = f'{drive}:/data/big2/ODS'
df_fold = f'{drive}:/data/big2/BD1/df'

# [로드] 등록정보&제원정보(표준화 위해)
- 필수 컬럼 추출
- 컬럼명 표준화

In [4]:
# about 8m 43.5s
name = '[ODS]등록정보&제원정보(2022.12.28)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29534047 entries, 0 to 29534046
Data columns (total 49 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   현재회원번호        object 
 7   차대번호          object 
 8   최초등록일자        int64  
 9   차량연식          int64  
 10  차량제작일자        float64
 11  배출가스검사유효일자    float64
 12  제원관리번호        object 
 13  배출가스인증번호      object 
 14  배출가스등급        object 
 15  저공해조치YN(미사용)  object 
 16  저공해조치종류       float64
 17  저공해조치구조변경YN   object 
 18  차량등록번호숫자정보    int64  
 19  차량말소YN        object 
 20  최초회원번호        object 
 21  취득일자          object 
 22  취득금액          float64
 23  생성일시_등록정보     object 
 24  수정일시_등록정보     object 
 25  차명            object 
 26  차종_제원정보       object 
 27  차종분류          object 
 28  차종유형(소분류)     object 
 29  자동차형식         object 
 30  제작사명          ob

In [5]:
df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보'],
      dtype='object')

In [6]:
df['차종_등록정보'].unique()

array(['승용', '화물', '승합', '특수', nan], dtype=object)

In [7]:
df['차종_등록정보'].isnull().sum()

14

In [8]:
df['용도'].unique()

array(['자가용', '영업용', '관용'], dtype=object)

In [9]:
df['차종유형(소분류)'].unique()

array(['소형', '경형', '중형', '대형', nan], dtype=object)

In [10]:
df['배출가스등급'].value_counts(dropna=False)

배출가스등급
2      11803371
3       9890563
1       2870152
5       2378599
4       1898773
X        305339
T2       250282
T3       136844
T5           49
T4           43
NaN          32
Name: count, dtype: int64

In [12]:
df['배출가스등급'].unique()

array(['5', 'X', 'T5', '4', '3', 'T4', 'T3', '2', '1', 'T2', nan],
      dtype=object)

In [14]:
df.loc[df['배출가스등급'] == '1', '등급'] = 1
df.loc[(df['배출가스등급'] == '2') | (df['배출가스등급'] == 'T2'), '등급'] = 2
df.loc[(df['배출가스등급'] == '3') | (df['배출가스등급'] == 'T3'), '등급'] = 3
df.loc[(df['배출가스등급'] == '4') | (df['배출가스등급'] == 'T4'), '등급'] = 4
df.loc[(df['배출가스등급'] == '5') | (df['배출가스등급'] == 'T5'), '등급'] = 5
df.loc[df['배출가스등급'] == 'X', '등급'] = 'X'
df.loc[df['배출가스등급'].isnull() == True, '등급'] = np.nan

In [15]:
df['등급'].value_counts(dropna=False)

등급
2.0    12053653
3.0    10027407
1.0     2870152
5.0     2378648
4.0     1898816
X        305339
NaN          32
Name: count, dtype: int64

In [16]:
df.shape

(29534047, 50)

In [17]:
df1 = df[[
    '차대번호',
    '본거지법정동코드',
    '연료', 
    '등급',
    '차종_등록정보',
    ]]
df1.shape

(29534047, 5)

In [18]:
cc_col = {
    '차대번호':'VIN', 
    '본거지법정동코드':'BSPL_STDG_CD', 
    '연료':'FUEL_CD', 
    '등급':'EXHST_GAS_GRD_CD', 
    '차종_등록정보':'VHCTY_CD', 
}

In [19]:
df1 = df1.rename(columns=cc_col)
df1.columns

Index(['VIN', 'BSPL_STDG_CD', 'FUEL_CD', 'EXHST_GAS_GRD_CD', 'VHCTY_CD'], dtype='object')

## [출력] 등록&제원&지역 정보

In [21]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.05.26_PM.04.35.34'

In [22]:
# about 1m 50.1s
file_name = f'BD1_STD_BD_CAR_CURSTT({today_date}).csv'
df1.to_csv(os.path.join(df_fold, file_name), index=False)

# 코드 마지막